# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
!nvidia-smi


Thu Aug 28 21:14:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:07.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244', 
                                      # 🔽 关键参数：offload_folder（必须存在）
                                      offload_folder="./offload",  # 创建该目录：mkdir -p ./offload
                                      # 🔽 可选：限制最大显存使用（防止占满）
                                      max_memory={0: "10GiB", "cpu": "32GiB"},  # T4 最多用 10G 显存，其余卸载到 CPU
                                      )

/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [6]:
input_text = "解释下乾卦是什么？"

In [7]:
response, history = base_model.chat(tokenizer, query=input_text)

In [8]:
print(response)

乾卦是八卦之一，也是八宫图中的第一个卦。乾卦是由两个阴爻夹一个阳爻构成，象征着天、干燥、刚强、积极向上、行动力等。乾卦的卦辞是“元、亨、利、贞”，表示万事万物的本源和正宗，具有无穷的力量和广阔的意义。

乾卦的六爻分别代表不同的天气和状态，从初爻到六爻，依次是：初爻为 cloud（初），二爻为 light（二），三爻为 heavy（三），四爻为 wind（四），五爻为 firmness（五），六爻为 clear（六）。

乾卦的应用非常广泛，可以用来指导人们的行为和决策，也可以用来分析事物的状态和发展趋势。乾卦的出现，意味着事物正在走向积极向上的方向，但也需要注意避免过于刚强和固执。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [9]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，也是八宫图中的第二个卦。讼卦由两个阳爻夹一个阴爻构成，象征着诉讼、争端、诉讼、矛盾等。

讼卦的卦辞是“元、亨、利、贞”，表示万事万物的本源和正宗，具有无穷的力量和广阔的意义。但是，由于诉讼和争端的存在，这个卦也预示着可能会带来一些负面影响和困难。

在周易中，每个卦都有不同的含义和解释，而讼卦也不例外。它的六爻分别代表不同的方面和状态，从初爻到六爻，依次是：初爻为 cloud（初），二爻为 light（二），三爻为 heavy（三），四爻为 wind（四），五爻为 firmness（五），六爻为 clear（六）。

讼卦的出现，意味着事物正在面临诉讼和争端的可能性，也预示着这些争端可能会带来一些不利的影响。因此，在遇到讼卦时，我们应该首先冷静下来，尽可能通过和平的方式来解决争端，避免让争端进一步恶化。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [11]:
!ls -R models/

models/:
THUDM

models/THUDM:
chatglm3-6b-epoch3-20250828_204439

models/THUDM/chatglm3-6b-epoch3-20250828_204439:
adapter_config.json	   checkpoint-20  checkpoint-50
adapter_model.safetensors  checkpoint-30  checkpoint-60
checkpoint-10		   checkpoint-40  README.md

models/THUDM/chatglm3-6b-epoch3-20250828_204439/checkpoint-10:
adapter_config.json	   optimizer.pt  rng_state.pth	trainer_state.json
adapter_model.safetensors  README.md	 scheduler.pt	training_args.bin

models/THUDM/chatglm3-6b-epoch3-20250828_204439/checkpoint-20:
adapter_config.json	   optimizer.pt  rng_state.pth	trainer_state.json
adapter_model.safetensors  README.md	 scheduler.pt	training_args.bin

models/THUDM/chatglm3-6b-epoch3-20250828_204439/checkpoint-30:
adapter_config.json	   optimizer.pt  rng_state.pth	trainer_state.json
adapter_model.safetensors  README.md	 scheduler.pt	training_args.bin

models/THUDM/chatglm3-6b-epoch3-20250828_204439/checkpoint-40:
adapter_config.json	   optimizer.pt  rng_state.pth	trainer_st

In [12]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20250828_204439"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [13]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [14]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由六个阳爻组成，位于卦象的北方。它代表天，是天地的开始，象征着大吉大利。然而，在占卜中，它也预示着一些不利的因素。乾卦的核心哲学是：天行健，君子拟天，健始尚终，观天时，把握君子自己的道。', 'content': '\n乾卦的卦象是由两个阳爻夹一个阴爻构成，象征着天地的合一，大吉大利。然而，在占卜中，它也预示着一些不利的因素。在事业、经商、求名等方面，都需要谨慎行事，不可轻举妄动。但在决策中，健始尚终，坚持正道，可以帮助自身获得成功。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250828_204439）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [15]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
{'name': '讼卦是一个占卜卦象，在周易卦象中，它由上卦乾（天）和下卦坎（水）组成。乾卦代表天，坎卦代表水，两者结合形成一个卦象，象征着天与地相融，光明与黑暗相激。这一卦象预示着事物相互依存，互为因果，互为彼我。', 'content': '\n讼卦的核心哲学是：天行健，君子道其道，止于至善。意味着天一直行健，人们应该效仿天的行健，止于至善，从而实现良好的结局。然而，尽管天行健，但君子仍然需要道法自然，止于至善，才能实现良好的结局。\n\n在这个卦象中，旱的形成与天相对，表示天行健，而君子当涉水过河。因此，这一卦象预示着君子应当勇敢地迎接挑战，冒险去冒犯他人，以此来实现愿望。然而，Also, it serves as a reminder that to achieve success, one must be patient and humble, and trust in the right path. It also suggests that one should act with justice and righteousness to avoid disputes and conflicts. Finally, it suggests that one should be flexible and adaptable in difficult situations, as the answer to success is not always what one expects.'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250828_204439）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象，它由上卦乾（天）和下卦坎（水）组成。在这个卦象中，天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得

In [16]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
师卦是由坎卦（水）和坤卦（地）相叠而成。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250828_204439）：
[gMASK]sop 师卦是什么？ 师卦是由坎卦（水）和坤卦（地）相叠而成。它代表着军队的力量和军情的总指挥，预示着吉祥无灾。在周易中，师卦的解释象征着吉祥无灾，但需要注意，行动之前必须要有计划和领导。师卦的核心哲学是“宁缺毋滥”，即军队虽然众多，但必须谨慎行事，以免损失。另外，师卦也象征着困难重重，需要谨慎应对，勇敢担当，才能获得美好的结局。

在事业、经商、求名、婚恋等方面的决策中，师卦都代表着困难与挑战。因此，若想在事业、经商、求名等方面取得成功，必须若是领导，必须谨慎行事，若非领导，必须寻找合适的工作机会。在婚恋方面，师卦代表着谨慎行事，不能轻信他人，必须小心谨慎地寻找合适的对象。

师卦的核心哲学是：若非领导，若在求名、事业、经商方面，必须谨慎行事，若在婚恋方面，必须小心谨慎地寻找合适的对象。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [19]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [20]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是中国传统哲学中的一个卦象，由六个阳爻组成，中间一个阳爻实、一个阳爻虚，形成一个天圆地方的卦象。乾卦代表着天（乾为天）的消息，它代表了一个刚健、健行、刚健不屈的意境。', 'content': '\n乾卦象征天，代表着刚健、不屈不挠的精神，强调坚韧不拔的意志。在事业、经商、求名和决策等方面，乾卦都代表着顺利、顺利行事，具有刚健、不屈不挠的精神。在婚恋方面，乾卦代表着天圆地方，意味着顺利结束婚姻。\n\n在人生观、价值观方面，乾卦代表着天行健，强调健行、坚韧不屈的精神，提倡模擬天行，行健、坚忍、不屈。在人际关系方面，乾卦代表着天健，强调大吉大利，要求互相信任，互惠互利。\n\n总体来说，乾卦象征着天行健，强调刚健、不屈不挠的精神，在事业、经商、求名和决策方面具有大吉大利的作用。在婚恋方面，乾卦代表着天圆地方，尽管顺利，但也存在一些潜在的危机。在人际关系方面，乾卦代表着天健，强调大吉大利，要求互相信任，互惠互利。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [21]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
{'name': '卜卦问卜者，地水师卦是由坎卦（水）和坤卦（地）相叠而成。在周易中，这一卦象征着顺从天道的标记，预示着吉祥悔吝。君子观此卦象，应当顺应自然、适应环境、适应夹缝生存，而不能轻举妄动。地水师卦的运势起伏有因，需要坚持正义、勇敢行事，否则将会有凶险之灾。地水师卦的运势期限是在指南posts指南posts guided by theSpirits of the deceased。', 'content': '\n地水师卦的核心哲学是：顺从天道、适应环境、适应夹缝生存，而不能轻举妄动。君子应当坚持正义、勇敢行事，而不是冒险行事。地水师卦的取舍为：顺应天道，适应环境，顺应夹缝生存，坚持正义，勇敢行事，而非轻举妄动。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 抱歉，地水师卦并不是一个常见的卦象。在周易中，卦象是由两个卦相交而形成，地水师卦可能是您将地卦和坎卦相加得到的。地卦代表着地球，坎卦代表着云泽，两者相加代表着云泽之应。这一卦象预示着 Cloud and dew meet, and the cloud reflects the dew. This signifies that the earth is covered with snow, and the moisture is abundant. However, the卦象 also warns of the danger of over-reliance on the cloud and the dew, which can lead to flooding. In the context of卜卦，地水师卦象征着吉祥，但 also warns of potential danger.


In [22]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
它是周易中的一卦，由上卦乾（天）和下卦坎（水）组成。这一卦象征着天与水相违，但它也预示着解决纷争的可能。在占卜中，讼卦代表着当前的困境和冲突。对于使用者而言，它意味着需要借助天力来解救，但同时也必须谨慎行事。在事业、经商、求名等方面，这一卦都暗示着困难重重的挑战，但有所不同 degree 的解决可能性。 overall，讼卦鼓励人们勇敢面对困难，寻求解决办法。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象，它由上卦乾（天）和下卦坎（水）组成。在这个卦象中，天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [24]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [25]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是中国传统易学的卦象之一，由六个阳爻组成，中间一个阳爻实、一个虚。它象征着天宇的广大、刚健和健行。乾卦代表刚健不阿，冒险前进，敢作敢为，敢攀敢堕的卦象。在实际生活中，它在卦象中显示了行健不息，刚健不阿，勇敢前进的象数。乾卦 also represents the天宇广大，刚健和健行，敢作敢为，敢攀敢堕的卦象。在实际生活中，它应在卦象中显示了行健不息，刚健不阿，勇敢前进的卦象。乾卦的哲学内涵是：刚健不阿，正派活泼，勇敢前进。在事业、经商、求名、婚恋等方面的决策中，都应取此卦象。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [26]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
在周易中，地水师卦是一个由两个卦相组合而成的卦象，代表着的事物相互交织，需要慎重处理。这个卦象代表人生中的困难与挫折，但也有克服困难的可能性。对于经商，意味着面临挫折和困难，需要谨慎行事，不可急躁。对于恋爱，卦象显示情感复杂，需要慎重处理，不能急功近利。地水师卦的核心哲学是，人生经商过程中，必须慎重行事，不可急躁，但也有克服困难的可能性。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦

 equal，the，a，and，in，on，of，to，for，with，by，from，about，above，below，in，on，under，between，of，for，with，about，above，below，in，on，under，between，of，for，with，about，above，below，in，on，under，between，of，for，with，about，above，below，in，on，under，between，of，for，with，about，above，below，in，on，under，between，of，for，with，about，above，below，in，on，under，between，of，for，with，about，above，below，in，on，under，between，of，for，with，about，above，below，in，on，under，between，of，for，with，about，above，below，in，on，under，between，of，for，with，about，above，below，in，on，under，between，of，for，with，about，above，below，in，on，under，between，of，for，with，about，above，below，in，on，under，between，of，for，with，about，above，below，in，on，under，between，of，for，with，about，above，below，in，on，under，between，of，for，with，abo

In [27]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
讼卦是周易中的一卦，由上卦乾（天）和下卦坎（水）组成。在这个卦中，阳卦（天）和阴卦（水）相互融合，形成一个和谐的整体。因此，这是一个吉利的卦象，预示着枯木逢春，复得新生的景象。

讼卦的核心哲学是：天行健，君子专迷于道，科险喜争，占卜者应避免与尖刻之人心相争。然而，尽管吉象明显，但仍有危险和挫折。在事业上，应避免冒进， awaits恢复的机会。在君子生活中，应避免贪图眼前利益，而应致力于长期的规划。

总体而言，讼卦表示一个充满矛盾与斗争的卦象，对于事业与人生都有不同的解释。在事业上，强调谨慎与耐心，避免冒险与贪进， awaits恢复的机会；对于君子生活，强调避免贪图眼前利益，而应致力于长期的规划。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦 坎卦与坤卦相互对立，一个象征水，一个象征地。在周易中，这两个卦相互对立，形成一种相克关系。坎卦代表着云，云聚散为天地，地雷震动为君子，君子行于云中，君子之道像云一样。坤卦则是地雷震动，表示刚健健行，君子刚健而健行，像云一样，君子美在其中。两卦都各有四卦，分别为云卦、云中卦、地雷卦、震动卦。在象辞中，云卦和地雷卦分别代表 Cloud and Thunder，两者都预示着 Cloudy Thundery Weather（云中风暴）。云中卦和震动卦则分别代表着 Cloudless and Thunder，预示着云中风暴将消逝。坤卦的君子刚健而健行，像云一样，坤卦的卦象由地雷卦和云卦组成，象征着君子刚健而健行，像云一样。坎卦的卦象由云卦和坤卦相交而成，象征着云中风暴，预示着云中风暴将消逝。
